In [1]:
import requests
import string
from pprint import pprint
import re
import os
import tempfile
from bs4 import BeautifulSoup
from collections import Counter, OrderedDict
from operator import itemgetter
import numpy as np
hookRegex = re.compile(r'\[.*\]')
tireRegex = re.compile(r' -')
chars_to_remove = string.punctuation.replace('-', '') + '—„“»«'
print(chars_to_remove)
api_url = 'http://api.genius.com'
base_url = 'http://genius.com'
headers = {'Authorization': 'Bearer ' + open('./.bearer_token.txt', 'r').read().rstrip('\n')}

!"#$%&'()*+,./:;<=>?@[\]^_`{|}~—„“»«


In [29]:
def remove_unnecessary(textfile):
    return tireRegex.sub('', hookRegex.sub('',textfile).translate(str.maketrans('','',chars_to_remove)))

class Artist(object):
    
    def __init__(self, name):
        self.name = name
        
    artist_id = ''
    songlist = []
    
    def get_artist_id(self):
        print("Finding the ID of an artist " + self.name)
        search_url = api_url + '/search'
        data = {'q': self.name}
        response = requests.get(search_url, params=data, headers=headers)
        self.artist_id = response.json()['response']['hits'][0]['result']['primary_artist']['api_path']
        print("ID of an artist " + self.name + " is " + self.artist_id[9:])
        with open('./artists_id_list.txt', 'r+') as textfile:
            if self.artist_id in textfile.read():
                pass
            else:
                textfile.write(textfile.read() + '\n' + self.name + "  -  " + self.artist_id)
        return
    #artists/143013/songs?sort=popularity&per_page=50
    #artists/143013/songs?sort=popularity&per_page=50&page=3
    def get_songlist(self):
        list_of_songs = []
        print("Getting the list of songs")
        for page in range(1,4):
            res = requests.get(api_url + self.artist_id + '/songs?sort=popularity&per_page=50&page=' + str(page), headers=headers)
            print("Going through page " + str(page))
            for song in res.json()['response']['songs']:
#                 print(song['full_title'])
                if ("Ft." in song['full_title']) | ("Remix" in song['full_title']):
                    print(song['full_title'] + " is a feat, skipping")
                else:
                    song_url = song['url']
                    list_of_songs.append(song_url)
        self.songlist = list_of_songs
        return
    
    
    def write_lyrics_from_path(self):
        with open("./lyrics/" + self.name.replace(' ', '_') + '_lyrics.txt', 'w') as lyricsfile:
            for song in self.songlist:
                print(str(self.songlist.index(song)) + " Now scraping " + song[19:])
                page_url = song
                res = requests.get(page_url, headers=headers)
                res.text
                page = requests.get(page_url)
                html = BeautifulSoup(page.text, 'html.parser')
                lyrics = html.find(class_='lyrics').get_text()
                lyricsfile.write(remove_unnecessary(lyrics.lower()))
        return
    
    def perform_task(self, write_id=False):
        self.get_artist_id()
        self.get_songlist()
        self.write_lyrics_from_path()
        return


In [34]:
%%time
with open('./artists_source.txt', 'r') as id_list:
    for artist in id_list.readlines():
        the_artist = Artist(artist.rstrip("\n"))
        the_artist.perform_task(write_id=True)

Finding the ID of an artist oxxxymiron
ID of an artist oxxxymiron is 24422
Getting the list of songs
Going through page 1
12/04/15: Oxxxymiron vs. Johnyboy by Versus Battle (Ft. Johnyboy & Oxxxymiron) is a feat, skipping
Fata Morgana by Markul (Ft. Oxxxymiron) is a feat, skipping
Безумие (Madness) [Remix] by ЛСП (LSP) (Ft. Oxxxymiron) is a feat, skipping
Пора возвращаться домой (PVD) by БИ-2 (BI-2) (Ft. Oxxxymiron) is a feat, skipping
13/08/2017: Oxxxymiron vs. Слава КПСС by Versus Battle (Ft. Oxxxymiron & Слава КПСС (Slava KPSS)) is a feat, skipping
Больше Бена (Bolshe Bena) by Oxxxymiron (Ft. Охра (Ohra)) is a feat, skipping
Darkside by Oxxxymiron (Ft. MadChild) is a feat, skipping
Imperial by Porchy (Ft. ЛСП (LSP) & Oxxxymiron) is a feat, skipping
Бездыханным (Breathless) by Жак-Энтони (Jacques-Anthony) (Ft. Oxxxymiron) is a feat, skipping
Машина Прогресса (Progress Machine) by Ка-тет (Ka-tet) (Ft. Oxxxymiron) is a feat, skipping
Привет со дна  (Hello from the bottom) by Oxxxymiron 

KeyboardInterrupt: 

In [28]:
def get_percentage(words, total):
    return round(words*100/total, 2)

def ord_dict(input_dict):
    return OrderedDict(sorted(input_dict.items(), key = itemgetter(1), reverse = True))

class Lyric_file(object): 
    the_text = ''
    the_list = []
    the_counter = {}
    the_total = 0
    def __init__(self, location):
        self.location = location
        self.the_text = open(self.location, 'r').read()
        self.the_list = self.the_text.split()
        self.the_counter = Counter(self.the_list)
        self.the_total = len(self.the_list)
        
    def get_most_common(self, number):
        return self.the_counter.most_common(number)
    
    def get_most_common_percentage(self, number):
        result_list = []
        for word in self.get_most_common(number):
            result_list.append((word[0], get_percentage(word[1], len(self.get_list()))))
        return result_list
    
    def get_percentage_of_word(self, word):
        return get_percentage(self.the_counter[word],len(self.the_list))
    
    def get_vocab_size(self):
        return len(self.the_list)/len(self.the_counter)
    

In [10]:
list_tuple = {}
for artist in os.listdir('./lyrics/'):
    lyric = Lyric_file('./lyrics/' + artist)
    list_tuple[artist] = lyric.get_percentage_of_word("я")

pprint(ord_dict(list_tuple))

{'T-Fest_lyrics.txt': 5.22,
 'pharaoh_lyrics.txt': 4.97,
 'Bumble_Beezy_lyrics.txt': 4.14,
 'Тимати_lyrics.txt': 3.7,
 'Скриптонит_lyrics.txt': 3.28,
 'Хаски_lyrics.txt': 3.2,
 'ЛСП_lyrics.txt': 2.98,
 'guf_lyrics.txt': 2.91,
 'Эджей_lyrics.txt': 2.88,
 'Грязный_рамирес_lyrics.txt': 2.76,
 'макс_корж_lyrics.txt': 2.67,
 'Слава_КПСС_lyrics.txt': 2.65,
 'SD_aka_Sadist_lyrics.txt': 2.54,
 'oxxxymiron_lyrics.txt': 2.53,
 'Big_russian_boss_lyrics.txt': 2.46,
 'Птаха_lyrics.txt': 2.36,
 'rickey_f_lyrics.txt': 2.36,
 'Кровосток_lyrics.txt': 2.08,
 'Каста_lyrics.txt': 1.85,
 'ноггано_lyrics.txt': 1.67,
 'ATL_lyrics.txt': 0.0}


In [65]:
list_tuple = {}
for artist in os.listdir('./lyrics/'):
    lyric = Lyric_file('./lyrics/' + artist)
    list_tuple[artist] = lyric.get_vocab_size()
pprint(OrderedDict(sorted(list_tuple.items(), key = itemgetter(1), reverse = True)))
    

{'ATL_lyrics.txt': 10.422311827956989,
 'T-Fest_lyrics.txt': 4.238836967808931,
 'Скриптонит_lyrics.txt': 4.186169134049888,
 'pharaoh_lyrics.txt': 4.035280047837353,
 'Bumble_Beezy_lyrics.txt': 4.022329589971272,
 'ЛСП_lyrics.txt': 3.9103348358016743,
 'Тимати_lyrics.txt': 3.746849315068493,
 'guf_lyrics.txt': 3.7419499910055767,
 'SD_aka_Sadist_lyrics.txt': 3.517179487179487,
 'Эджей_lyrics.txt': 3.0998680158380996,
 'макс_корж_lyrics.txt': 3.083668543845535,
 'Каста_lyrics.txt': 3.0067528458421764,
 'Хаски_lyrics.txt': 2.8574816220061656,
 'oxxxymiron_lyrics.txt': 2.787449655710017,
 'Big_russian_boss_lyrics.txt': 2.757631578947368,
 'ноггано_lyrics.txt': 2.6236297198538368,
 'Кровосток_lyrics.txt': 2.4446957236842106,
 'Грязный_рамирес_lyrics.txt': 2.4247372675828616,
 'Слава_КПСС_lyrics.txt': 2.263233190271817,
 'Птаха_lyrics.txt': 2.1091617933723197,
 'rickey_f_lyrics.txt': 2.1091617933723197}


In [22]:
list_of_me = ['я', "мой", "моя", "мои", "меня", "мы", "наш"]
lyricobj = Lyric_file('./lyrics/guf_lyrics.txt')

In [23]:
total = 0
for me in list_of_me:
    total += lyricobj.the_counter[me]
print(total)
print(len(lyricobj.the_list))
print(len(lyricobj.the_list)/total)

1839
41603
22.6226209896683


In [25]:
with open('./something.txt', 'r+') as txt:
    if "Hallo" in txt.read():
        txt.write(txt.read() + "Somethign\n")
    else:
        txt.write(txt.read() + "Hllo\n")
    print(txt.read())
    

In [33]:
!ls
!cat artists_id_list.txt

artists_id_list.txt  Genius_version2.ipynb  README.md
artists_source.txt   lyrics

oxxxymiron  -  /artists/24422
Каспийский груз  -  /artists/1042099
guf  -  /artists/34707
Грязный рамирес  -  /artists/323240
Слава КПСС  -  /artists/1148720
ЛСП  -  /artists/143013
Скриптонит  -  /artists/154390
Каста  -  /artists/234245
Кровосток  -  /artists/233193
Эджей  -  /artists/662644
ATL  -  /artists/68978
ноггано  -  /artists/1048629
Big russian boss  -  /artists/161760
Bumble Beezy  -  /artists/215010
Птаха  -  /artists/349224
pharaoh  -  /artists/101583
макс корж  -  /artists/1050560
Тимати  -  /artists/20382
T-Fest  -  /artists/1063927
SD aka Sadist  -  /artists/1039745
Хаски  -  /artists/64942
Фэйс  -  /artists/988966